In [22]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

data_file = '/Users/devshah/Documents/WorkSpace/University/year 3/CSC392/Trading_Simulator/data/data.csv'
df = pd.read_csv(data_file, parse_dates=True, index_col=0)

feature_columns = [col for col in df.columns if col != 'target']
X = df[feature_columns].values
y = df['target'].values

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

split_index = int(0.8 * len(X_scaled))
X_train = X_scaled[:split_index]
X_test = X_scaled[split_index:]
y_train = y[:split_index]
y_test = y[split_index:]

# noise = np.random.normal(0, 0.01, X_train.shape)
# X_train = X_train + noise


X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32).unsqueeze(1)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32).unsqueeze(1)

batch_size = 32
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

class TradingNet(nn.Module):
    def __init__(self, input_dim):
        super(TradingNet, self).__init__()
        self.net = nn.Sequential(
            nn.Linear(input_dim, 128),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(64, 32),
            nn.ReLU(),
            nn.Linear(32, 1),
            nn.Sigmoid()
        )
        
    def forward(self, x):
        return self.net(x)

input_dim = X_train.shape[1]
model = TradingNet(input_dim).to(device)

criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-5)


def evaluate(model, X_tensor, y_tensor):
    model.eval()
    with torch.no_grad():
        outputs = model(X_tensor.to(device))
        loss = criterion(outputs, y_tensor.to(device))
        preds = (outputs > 0.5).float()
        accuracy = (preds.eq(y_tensor.to(device)).sum().item()) / len(y_tensor)
    return loss.item(), accuracy

epochs = 100
for epoch in range(1, epochs + 1):
    model.train()
    running_loss = 0.0
    for batch_X, batch_y in train_loader:
        batch_X, batch_y = batch_X.to(device), batch_y.to(device)
        outputs = model(batch_X)
        loss = criterion(outputs, batch_y)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * batch_X.size(0)
    
    epoch_loss = running_loss / len(train_loader.dataset)
    
    if epoch % 10 == 0 or epoch == 1:
        train_loss_epoch, train_acc_epoch = evaluate(model, X_train_tensor, y_train_tensor)
        print(f"Epoch {epoch}/{epochs} - Loss: {epoch_loss:.4f} - Training Accuracy: {train_acc_epoch:.2f}")

train_loss, train_acc = evaluate(model, X_train_tensor, y_train_tensor)
test_loss, test_acc = evaluate(model, X_test_tensor, y_test_tensor)
print(f"\nFinal Training Loss: {train_loss:.4f}, Training Accuracy: {train_acc:.2f}")
print(f"Final Test Loss: {test_loss:.4f}, Test Accuracy: {test_acc:.2f}")

model.eval()
with torch.no_grad():
    y_pred_prob = model(X_test_tensor.to(device))
    y_pred = (y_pred_prob > 0.5).float().cpu().numpy().flatten()

if 'close' in df.columns:
    analysis_df = df.iloc[split_index:].copy()
    analysis_df['predicted_direction'] = y_pred
    analysis_df['next_day_return'] = analysis_df['close'].pct_change().shift(-1)
    analysis_df['strategy_return'] = analysis_df['next_day_return'] * analysis_df['predicted_direction']
    analysis_df['cumulative_strategy_return'] = (1 + analysis_df['strategy_return']).cumprod()
    analysis_df['cumulative_market_return'] = (1 + analysis_df['next_day_return']).cumprod()
    strategy_return = analysis_df['cumulative_strategy_return'].iloc[-2] - 1
    market_return = analysis_df['cumulative_market_return'].iloc[-2] - 1
    print(f"Strategy Return: {strategy_return:.2%}")
    print(f"Market Return: {market_return:.2%}")


Using device: cpu
Epoch 1/100 - Loss: 0.6918 - Training Accuracy: 0.52
Epoch 10/100 - Loss: 0.6684 - Training Accuracy: 0.61
Epoch 20/100 - Loss: 0.6324 - Training Accuracy: 0.67
Epoch 30/100 - Loss: 0.5941 - Training Accuracy: 0.71
Epoch 40/100 - Loss: 0.5721 - Training Accuracy: 0.74
Epoch 50/100 - Loss: 0.5298 - Training Accuracy: 0.78
Epoch 60/100 - Loss: 0.5214 - Training Accuracy: 0.80
Epoch 70/100 - Loss: 0.5064 - Training Accuracy: 0.81
Epoch 80/100 - Loss: 0.4939 - Training Accuracy: 0.83
Epoch 90/100 - Loss: 0.5046 - Training Accuracy: 0.84
Epoch 100/100 - Loss: 0.4685 - Training Accuracy: 0.84

Final Training Loss: 0.3578, Training Accuracy: 0.84
Final Test Loss: 0.8265, Test Accuracy: 0.51
Strategy Return: -33.19%
Market Return: -28.49%


In [5]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
import torch.nn.functional as F

In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader

# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Load data
data_file = '/Users/devshah/Documents/WorkSpace/University/year 3/CSC392/Trading_Simulator/data/data.csv'
df = pd.read_csv(data_file, parse_dates=True, index_col=0)

feature_columns = [col for col in df.columns if col != 'target']
X = df[feature_columns].values
y = df['target'].values

# Standard scaling
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Train-test split
split_index = int(0.8 * len(X_scaled))
X_train = X_scaled[:split_index]
X_test = X_scaled[split_index:]
y_train = y[:split_index]
y_test = y[split_index:]

# Convert to tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32).unsqueeze(1)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32).unsqueeze(1)

batch_size = 32
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

# ------------------------------
# Updated Model with BatchNorm and Increased Dropout
# ------------------------------
class TradingNet(nn.Module):
    def __init__(self, input_dim):
        super(TradingNet, self).__init__()
        self.net = nn.Sequential(
            nn.Linear(input_dim, 128),
            nn.BatchNorm1d(128),
            nn.ReLU(),
            nn.Dropout(0.5),  # Increased dropout
            nn.Linear(128, 64),
            nn.BatchNorm1d(64),
            nn.ReLU(),
            nn.Dropout(0.5),  # Increased dropout
            nn.Linear(64, 32),
            nn.BatchNorm1d(32),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(32, 1),
            nn.Sigmoid()
        )
        
    def forward(self, x):
        return self.net(x)

input_dim = X_train.shape[1]
model = TradingNet(input_dim).to(device)

criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-5)

# ------------------------------
# Evaluation Function
# ------------------------------
def evaluate(model, X_tensor, y_tensor):
    model.eval()
    with torch.no_grad():
        outputs = model(X_tensor.to(device))
        loss = criterion(outputs, y_tensor.to(device))
        preds = (outputs > 0.5).float()
        accuracy = (preds.eq(y_tensor.to(device)).sum().item()) / len(y_tensor)
    return loss.item(), accuracy

# ------------------------------
# Training Loop with Early Stopping
# ------------------------------
epochs = 100
best_val_loss = float('inf')
patience = 10
trigger_times = 0

for epoch in range(1, epochs + 1):
    model.train()
    running_loss = 0.0
    for batch_X, batch_y in train_loader:
        batch_X, batch_y = batch_X.to(device), batch_y.to(device)
        outputs = model(batch_X)
        loss = criterion(outputs, batch_y)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * batch_X.size(0)
    
    epoch_loss = running_loss / len(train_loader.dataset)
    val_loss, val_acc = evaluate(model, X_test_tensor, y_test_tensor)
    
    # Early stopping check
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        trigger_times = 0
        # Optionally, save the model checkpoint here
    else:
        trigger_times += 1
        if trigger_times >= patience:
            print(f"Early stopping triggered at epoch {epoch}")
            break
    
    if epoch % 10 == 0 or epoch == 1:
        train_loss_epoch, train_acc_epoch = evaluate(model, X_train_tensor, y_train_tensor)
        print(f"Epoch {epoch}/{epochs} - Train Loss: {epoch_loss:.4f} - Train Acc: {train_acc_epoch:.2f} - Val Loss: {val_loss:.4f} - Val Acc: {val_acc:.2f}")

# ------------------------------
# Final Evaluation and Analysis
# ------------------------------
train_loss, train_acc = evaluate(model, X_train_tensor, y_train_tensor)
test_loss, test_acc = evaluate(model, X_test_tensor, y_test_tensor)
print(f"\nFinal Training Loss: {train_loss:.4f}, Training Accuracy: {train_acc:.2f}")
print(f"Final Test Loss: {test_loss:.4f}, Test Accuracy: {test_acc:.2f}")

model.eval()
with torch.no_grad():
    y_pred_prob = model(X_test_tensor.to(device))
    y_pred = (y_pred_prob > 0.5).float().cpu().numpy().flatten()

if 'close' in df.columns:
    analysis_df = df.iloc[split_index:].copy()
    analysis_df['predicted_direction'] = y_pred
    analysis_df['next_day_return'] = analysis_df['close'].pct_change().shift(-1)
    analysis_df['strategy_return'] = analysis_df['next_day_return'] * analysis_df['predicted_direction']
    analysis_df['cumulative_strategy_return'] = (1 + analysis_df['strategy_return']).cumprod()
    analysis_df['cumulative_market_return'] = (1 + analysis_df['next_day_return']).cumprod()
    strategy_return = analysis_df['cumulative_strategy_return'].iloc[-2] - 1
    market_return = analysis_df['cumulative_market_return'].iloc[-2] - 1
    print(f"Strategy Return: {strategy_return:.2%}")
    print(f"Market Return: {market_return:.2%}")


Using device: cpu
Epoch 1/100 - Train Loss: 0.7045 - Train Acc: 0.54 - Val Loss: 0.6922 - Val Acc: 0.53
Epoch 10/100 - Train Loss: 0.6891 - Train Acc: 0.57 - Val Loss: 0.6907 - Val Acc: 0.54
Early stopping triggered at epoch 20

Final Training Loss: 0.6703, Training Accuracy: 0.59
Final Test Loss: 0.6918, Test Accuracy: 0.53
Strategy Return: 437.57%
Market Return: -28.49%
